# "Remote Jupyter Notebook"

> "setting up jupyter notebooks remotely"
- toc: true
- branch: master
- badges: false
- comments: true
- categories: [Others]
- hide: false
- search_exclude: false
- image: images/post-thumbnails/jn.png
- metadata_key1: JupyterNotebook
- metadata_key2: 

## Setup Remote Jupyter Notebook

Useful in cases where you want to access jupyter notebooks from MAC or WINDOWS but want to setup UBUNTU servers with jupyter notebooks. 


### CREATE EXEC SCRIPT

- CREATE jupyter-startup.sh 

```bash
source ~/miniconda3/etc/profile.d/conda.sh
conda activate YOUR_CONDA_VIRTUAL_ENVIRONMENT
jupyter notebook
``` 

### CREATE JUPYTER SERVICE

- CREATE a following jupyter service in your home directory
- COPY the file to this destination ==> **/etc/systemd/system/jupyter.service**

```bash

[Unit]

Description=Jupyter Notebook Service Description

[Service]

Type=simple

PIDFile=/run/jupyter.pid

ExecStart=/home/YOUR_USER_NAME/jupyter-startup.sh 

User=YOUR_USER_NAME

Group=YOUR_USER_NAME

Restart=always

RestartSec=10

[Install]

WantedBy=multi-user.target

```

```bash
- sudo systemctl enable jupyter.service
- sudo systemctl daemon-reload
- sudo systemctl restart jupyter.service
```

> Important : Pick either option 1 or 2 (but not both).

### SSH Tunnel (option 1)


1) Setup a password to the notebook server so that we dont have to provide token every single time. On the remote server, do the following

```bash

jupyter notebook password
#Enter password:  ****
#Verify password: ****
#[NotebookPasswordApp] Wrote hashed password to /Users/<<you>>/.jupyter/jupyter_notebook_config.json

```

2) Run SSH tunnel to setup a link on the local host 

Pick any port number for LOCAL_PORT. You may need to enter a passphrase.

```bash

ssh -N -f -L localhost:<LOCAL_PORT>:localhost:8080 username@remotemachine -p <<PORT ID>> -v  

```

3) Type http://localhost:8888/tree in your local browser to access the jupyter notebook setup remotely. 

--------------------------------------------------------------------------------------------------------------

### PUBLIC URL (option 2)

#### CONFIG JUPYTER NOTEBOOK

- Create Jupyter Notebook Configuration

```bash

jupyter notebook –generate-config

```

##### CREATE PASSWORD and CERTIFICATES

- First generate a new password to login

In [1]:
from notebook.auth import passwd
import IPython

passwd()


Enter password: ········
Verify password: ········


'sha1:6ed7a3ec3487:57c304ac1ba182fa7c160d2fcac788a2c376cb5f'

 
 - **Generate the certificate file. It will be used in the configuration step**
 
```bash
openssl req -x509 -nodes -days 365 -newkey rsa:2048 -keyout mykey.key -out mycert.pem
```
 
![](https://abhisheksreesaila.github.io/blog/images/general/cert-gen.png "Certificate Generation")

 
##### EDIT  NOTEBOOK CONFIGURATION FILES

```bash
cd .jupyter
nano jupyter_notebook_config.py
```

- **Open the configuration file and edit as follows**

  - c.NotebookApp.allow_origin = '*'
  - c.NotebookApp.allow_remote_access = True
  - c.NotebookApp.certfile = 'YOUR_CERTIFICATE_FILE'
  - c.NotebookApp.ip = '0.0.0.0'
  - c.NotebookApp.open_browser = False
  - c.NotebookApp.password = u'<ENTER YOUR PASSWORD FROM ABOVE>'
  - c.NotebookApp.port = ANY_PORT_YOU_LIKE


#### EDIT FIREWALL RULES (if needed)
```bash
    sudo ufw allow from <<IP ADDRESS>>
```
    

# References

- [Config Jupyter Notebook](https://jupyter-notebook.readthedocs.io/en/stable/public_server.html)
- [Config Jupyter Notebook 2](https://hyunyoung2.github.io/2017/11/14/How_to_Access_Jupyter_Notebook_Remotely)
- [Create Jupyter Service](https://gist.github.com/whophil/5a2eab328d2f8c16bb31c9ceaf23164f)
- [Create Jupyter Service 2](https://janakiev.com/blog/jupyter-systemd)
- [Configure Linux firewall](https://www.linode.com/docs/guides/configure-firewall-with-ufw)
